In [5]:
import pandas as pd

df_summary = pd.read_csv('parsed_summary.csv')

In [97]:
#these are values around 10!
df_10 = df_summary[df_summary["cluster"] == 0]
df_0 = df_summary[df_summary["cluster"] == 1]

In [115]:
import pickle 
import GEOparse

bsu_to_gene = pickle.load( open( "bsu_to_gene.dict", "rb" ) )
gene_to_bsu = pickle.load( open( "gene_to_bsu.dict", "rb" ) )

GEOparse.set_verbosity('ERROR')

def read_experiment(experiment):
    return GEOparse.get_GEO(filepath="data/" + experiment + "_family.soft.gz")

def make_data_frame(gsm):
    df = pd.DataFrame(index=list(bsu_to_gene))
    for sample in list(gsm.gsms):
        description = " ".join((gsm.gsms[sample].metadata["title"][0], "("+gsm.metadata["geo_accession"][0]+")"))
        table = gsm.gsms[sample].table
        table = table.loc[table['ID_REF'].isin(list(bsu_to_gene)) | table['ID_REF'].isin(list(gene_to_bsu))]
        table = table.replace({"ID_REF": gene_to_bsu}) 
        table = table.set_index("ID_REF")
        table = table.rename(columns={'VALUE': description})
        df = df.join(table, lsuffix="_l", rsuffix="_r")
    return df

df = pd.DataFrame(index=list(bsu_to_gene))
for ex in df_10["series_accession"].values:
    gsm = read_experiment(ex)
    df = df.join(make_data_frame(gsm),lsuffix="_l", rsuffix="_r")

C:\Users\pim_d\Anaconda3\lib\site-packages\GEOparse\GEOparse.py:332: FutureWarning:

from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls



In [116]:
df.fillna(0)
df.to_csv("rmna_value_matrix.csv", encoding='utf-8', index=True)

In [109]:
# interactive heatmap
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
#plotly.tools.set_credentials_file(username='mcx2576', api_key='FbmPqzV9kLPnpkW59Lgk')
colorscale = [[0, '#4dac26'], [.25, '#b8e186'],  [.5, '#f7f7f7'],  [.75, '#f1b6da'],  [1, '#d01c8b']]

heatmap = go.Heatmap(z=df.values, x=df.columns, y=df.index, colorscale=colorscale)
data = [heatmap]
py.iplot(data, filename='basic-heatmap')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~mcx2576/0 or inside your plot.ly account where it is named 'basic-heatmap'


In [110]:
df = pd.DataFrame(index=list(bsu_to_gene))
for ex in df_0["series_accession"].values:
    gsm = read_experiment(ex)
    df = df.join(make_data_frame(gsm),lsuffix="_l", rsuffix="_r")

C:\Users\pim_d\Anaconda3\lib\site-packages\GEOparse\GEOparse.py:332: FutureWarning:

from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls



In [111]:
df = df.fillna(0)

In [113]:
# interactive heatmap
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
#plotly.tools.set_credentials_file(username='mcx2576', api_key='FbmPqzV9kLPnpkW59Lgk')
colorscale = [[0, '#4dac26'], [.25, '#b8e186'],  [.5, '#f7f7f7'],  [.75, '#f1b6da'],  [1, '#d01c8b']]

heatmap = go.Heatmap(z=df.values, x=df.columns, y=df.index, colorscale=colorscale, zmin=-15, zmax=15)
data = [heatmap]
py.iplot(data, filename='basic-heatmap')

PlotlyRequestError: 
<html><head>
<meta http-equiv="content-type" content="text/html;charset=utf-8">
<title>502 Server Error</title>
</head>
<body text=#000000 bgcolor=#ffffff>
<h1>Error: Server Error</h1>
<h2>The server encountered a temporary error and could not complete your request.<p>Please try again in 30 seconds.</h2>
<h2></h2>
</body></html>


In [114]:
df.to_csv("fold_change_matrix.csv", encoding='utf-8', index=True)